## Hyperparameter Tuning

In our pursuit of optimizing predictive performance for California housing price prediction, we turn our attention towards hyperparameter tuning. 

Hyperparameters play a pivotal role in shaping the behavior and performance of machine learning models, and fine-tuning them can lead to significant improvements in predictive accuracy and generalization.

#### Loading and preparing the data

In [2]:
from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [3]:
california = fetch_california_housing()
print(california["DESCR"])

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [4]:
df_cali = pd.DataFrame(california["data"], columns = california["feature_names"])
df_cali["median_house_value"] = california["target"]

df_cali.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,median_house_value
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


#### Normalization & Feature Selection

Like we did in Feature Engineering lesson, we are going to normalize our data and select a subset of columns as our features.

#### Train Test Split

In [5]:
features = df_cali.drop(columns = ["median_house_value","AveOccup", "Population", "AveBedrms"])
target = df_cali["median_house_value"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

Create an instance of the normalizer

In [7]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

MinMaxScaler()

In [8]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [9]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

# Grid Search

**Grid Search** - we define a grid of hyperparameter values we want to try. Grid Search tries all possible combinations.

So far, our best model was AdaBoost yield a R-Squared of 0.83.


Let's see how we fine tune our model, in order to that, we will optimize the following hyperparameters:

- **n_estimators:** number of estimators, in this case, number of trees

- **max_leaf_nodes:** maxium number of total leafs to consider

- **max_depth:** maxium number of levels in each tree

- First we define the grid with values to consider when train all possible combinations.

In [10]:
grid = {"n_estimators": [50, 100],
        "estimator__max_depth":[5,10]}

In [11]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor())

In [12]:
model = GridSearchCV(estimator = ada_reg, param_grid = grid, cv=5)

In [13]:
model.fit(X_train_norm, y_train)

GridSearchCV(cv=5,
             estimator=AdaBoostRegressor(estimator=DecisionTreeRegressor()),
             param_grid={'estimator__max_depth': [5, 10],
                         'n_estimators': [50, 100]})

- After training, we check what are the best values for the hyperparameters that we have tested.

In [14]:
model.best_params_

{'estimator__max_depth': 10, 'n_estimators': 100}

- You can retrieve the best model with the best parameters when accessing **best_estimator_** attribute

In [15]:
best_model = model.best_estimator_

In [17]:
best_model

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=10),
                  n_estimators=100)

- Evaluate our model

In [19]:
pred = best_model.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", best_model.score(X_test_norm, y_test))

MAE 0.40448042348438484
RMSE 0.5309476881608578
R2 score 0.7838073639947809


### Training the final model

In [62]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 10), n_estimators = 100)

ada_reg.fit(X_train_norm,y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=10),
                  n_estimators=100)

# Random Search

**Random Search** - we define probability distributions for each hyperparameter, from which random values are sampled. It’s up to the researcher to set the maximum number of combinations.

In [69]:
grid = {"n_estimators": [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
        "estimator__max_leaf_nodes": [int(x) for x in np.linspace(start = 500, stop = 3000, num = 10)],
        "estimator__max_depth":[int(x) for x in np.linspace(10, 110, num = 11)]}

In [71]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor())

model = RandomizedSearchCV(estimator = ada_reg, param_distributions = grid, n_iter = 10, cv = 5, n_jobs = -1)

In [72]:
model.fit(X_train_norm,y_train)

RandomizedSearchCV(cv=5,
                   estimator=AdaBoostRegressor(estimator=DecisionTreeRegressor()),
                   n_jobs=-1,
                   param_distributions={'estimator__max_depth': [10, 20, 30, 40,
                                                                 50, 60, 70, 80,
                                                                 90, 100, 110],
                                        'estimator__max_leaf_nodes': [500, 777,
                                                                      1055,
                                                                      1333,
                                                                      1611,
                                                                      1888,
                                                                      2166,
                                                                      2444,
                                                                      2722,
                                                                      3000],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]})

In [73]:
model.best_params_

{'n_estimators': 1400,
 'estimator__max_leaf_nodes': 3000,
 'estimator__max_depth': 60}

- You can retrieve the best model with the best parameters when accessing **best_estimator_** attribute

In [74]:
best_model = model.best_estimator_

- Evaluate our model

In [75]:
pred = best_model.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", best_model.score(X_test_norm, y_test))

MAE 0.2900017048939809
RMSE 0.4638889040809561
R2 score 0.8349690507337173


We dont guarantee these hyperparameters are optimal! We can just guarantee that these are the best from the ones we tried!

### Using gridsearch find the optimal value for n_neighbors for this dataset

In [35]:
grid = {"n_neighbors": list(range(0,40,5))}

In [38]:
from sklearn.neighbors import KNeighborsRegressor

In [39]:
{'n_neighbors': 10}

In [63]:
model = GridSearchCV(estimator = knn, param_grid = grid, cv=5, verbose = 3)

In [64]:
model.fit(X_train_norm,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END .......................n_neighbors=0;, score=nan total time=   0.0s
[CV 2/5] END .......................n_neighbors=0;, score=nan total time=   0.0s
[CV 3/5] END .......................n_neighbors=0;, score=nan total time=   0.0s
[CV 4/5] END .......................n_neighbors=0;, score=nan total time=   0.0s
[CV 5/5] END .......................n_neighbors=0;, score=nan total time=   0.0s
[CV 1/5] END .....................n_neighbors=5;, score=0.702 total time=   0.0s
[CV 2/5] END .....................n_neighbors=5;, score=0.714 total time=   0.0s
[CV 3/5] END .....................n_neighbors=5;, score=0.682 total time=   0.0s
[CV 4/5] END .....................n_neighbors=5;, score=0.709 total time=   0.0s
[CV 5/5] END .....................n_neighbors=5;, score=0.701 total time=   0.0s
[CV 1/5] END ....................n_neighbors=10;, score=0.701 total time=   0.0s
[CV 2/5] END ....................n_neighbors=10;,

C:\Users\Aleks\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
5 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Aleks\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Aleks\anaconda3\lib\site-packages\sklearn\neighbors\_regression.py", line 215, in fit
    self._validate_params()
  File "C:\Users\Aleks\anaconda3\lib\site-packages\sklearn\base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\Aleks\anaconda3\lib\site-pack

GridSearchCV(cv=5, estimator=KNeighborsRegressor(n_neighbors=10),
             param_grid={'n_neighbors': [0, 5, 10, 15, 20, 25, 30, 35]},
             verbose=3)

In [42]:
model.best_params_

{'n_neighbors': 10}

In [59]:
knn = KNeighborsRegressor(**{'n_neighbors': 10})

In [60]:
knn.fit(X_train_norm, y_train)

KNeighborsRegressor(n_neighbors=10)

In [61]:
knn.score(X_test_norm,y_test)

0.7040267084262604